# Iterative Linear Solvers

This notebook covers iterative methods for solving systems of linear equations of the form $Ax = b$.

## Categories
1. **Stationary Iterative Methods**: Jacobi, Gauss-Seidel, Successive Over-Relaxation (SOR). These methods involve fixed-point iteration.
2. **Krylov Subspace Methods**: Conjugate Gradient (CG), GMRES. These methods minimize the residual over a Krylov subspace.

In [8]:
import numpy as np
import numpy.linalg as lng
import math
import matplotlib.pyplot as plt
import pandas as pd

## 1. Stationary Iterative Methods

### Utilities

In [9]:
def split_matrix(A):
    """
    Splits matrix A into its Strict Lower (L), Diagonal (D), and Strict Upper (U) components.
    Note: A = L + D + U
    """
    n = A.shape[0] 
    L = np.zeros((n, n))
    D = np.zeros((n, n))
    U = np.zeros((n, n))

    for i in range(0, n):
        L[i+1:, i] = A[i+1:, i]
        D[i, i]    = A[i, i]
        U[i, i+1:] = A[i, i+1:]
    return L, D, U

def spectral_radius(T):
    """
    Computes the spectral radius of the iteration matrix T.
    Necessary condition for convergence: rho(T) < 1.
    """
    radius = np.amax(abs(np.linalg.eigvals(T)))
    # print(f'Spectral Radius = {radius}')
    
    if radius >= 1:
        print(f"Warning: Spectral Radius {radius:.4f} >= 1. Method might not converge.")
    return radius

### Jacobi Method
$$x^{(k+1)} = D^{-1}(b - (L+U)x^{(k)})$$

In [10]:
def jacobi(A, b, RTOL=1e-6, ATOL=1e-6):
    L, D, U = split_matrix(A)
    S = -L - U
    T = np.dot(lng.inv(D), S)
    
    # Check convergence condition
    spectral_radius(T)
    
    c = np.dot(lng.inv(D), b)

    x = np.ones(A.shape[0])
    x_next = np.zeros(A.shape[0])
    error = lng.norm(x - x_next)
    iteration = 0

    while error > RTOL * lng.norm(x) + ATOL:
        x_next = np.dot(T, x) + c
        error = lng.norm(x - x_next)
        x = x_next
        iteration += 1
        if iteration > 1000:
            print("Max iterations reached")
            break
            
    return x_next, iteration

### Gauss-Seidel Method
$$x^{(k+1)} = (D+L)^{-1}(b - Ux^{(k)})$$

In [11]:
def gauss_seidel(A, b, RTOL=1e-6, ATOL=1e-6):
    L, D, U = split_matrix(A)
    T = np.dot(-lng.inv(D + L), U)
    
    spectral_radius(T)
    
    c = np.dot(lng.inv(D + L), b)
    
    x = np.ones(A.shape[0])
    x_next = np.zeros(A.shape[0])
    error = lng.norm(x - x_next)
    iteration = 0
    
    while error > RTOL * lng.norm(x) + ATOL:
        x_next = np.dot(T, x) + c
        error = lng.norm(x - x_next)
        x = x_next
        iteration += 1
        if iteration > 1000:
            print("Max iterations reached")
            break
            
    return x_next, iteration

### Successive Over-Relaxation (SOR)
Enhancement of Gauss-Seidel with relaxation parameter $\omega$.

In [12]:
def sor(A, b, w, RTOL=1e-6, ATOL=1e-6):
    L, D, U = split_matrix(A)
    
    # T = (D + wL)^(-1) * ((1-w)D - wU)
    T = np.dot(lng.inv(D + w * L), ((1 - w) * D - w * U))
    
    spectral_radius(T)
    if w >= 2:
        raise Exception("SOR method won't converge for w >= 2")
    
    # c = (D + wL)^(-1) * w*b
    c = np.dot(lng.inv(D + w * L), w * b)
    
    x = np.ones(A.shape[0])
    x_next = np.zeros(A.shape[0])
    error = lng.norm(x - x_next)
    iteration = 0
    
    while error > RTOL * lng.norm(x) + ATOL:
        x_next = np.dot(T, x) + c
        error = lng.norm(x - x_next)
        x = x_next
        iteration += 1
        if iteration > 1000:
            print("Max iterations reached")
            break
            
    return x_next, iteration

## 2. Krylov Subspace Methods

### Conjugate Gradient (CG)
Applicable for symmetric positive-definite matrices.

In [13]:
def conjugate_gradient(A, b, x0=None, tol=1e-5, max_iter=None):
    """
    Solves Ax = b using the Conjugate Gradient method.
    A must be symmetric positive-definite.
    """
    n = len(b)
    if x0 is None:
        x0 = np.zeros(n)
    if max_iter is None:
        max_iter = n * 10
        
    x = x0.copy()
    r = b - A.dot(x)
    p = r.copy()
    rsold = np.dot(r, r)
    
    for i in range(max_iter):
        Ap = A.dot(p)
        alpha = rsold / np.dot(p, Ap)
        x = x + alpha * p
        r = r - alpha * Ap
        rsnew = np.dot(r, r)
        
        if np.sqrt(rsnew) < tol:
            return x, i+1
            
        p = r + (rsnew / rsold) * p
        rsold = rsnew
        
    print('CG did not converge')
    return x, max_iter

### Comparison Test

In [35]:
# Test System
# Adjusting A to be SPD for CG and Diagonally Dominant for others
A_spd = np.array([[4, 1, 1], [1, 4, 1], [1, 1, 4]], dtype=float)
b = np.array([12, 3, 7], dtype=float)

results = []

import pandas as pd

solvers = [
    ("Jacobi", lambda: jacobi(A_spd, b)),
    ("Gauss-Seidel", lambda: gauss_seidel(A_spd, b)),
    ("SOR (w=1.1)", lambda: sor(A_spd, b, w=1.1)),
    ("Conjugate Gradient", lambda: conjugate_gradient(A_spd, b))
]

for name, solve in solvers:
    try:
        x, it = solve()
        results.append({"Method": name, "Solution": x, "Iterations": it, "Converged": True})
    except Exception as e:
        results.append({"Method": name, "Solution": str(e), "Iterations": "-", "Converged": False})

pd.set_option('display.max_colwidth', None)
display(pd.DataFrame(results))

,Method,Solution,Iterations,Converged
0,Jacobi,"[2.7777782016273704, -0.22222179836171563, 1.111111534966767]",19,True
1,Gauss-Seidel,"[2.777777702451999, -0.22222215222367936, 1.11111111244292]",8,True
2,SOR (w=1.1),"[2.777777218378912, -0.22222221251994195, 1.1111112773855905]",8,True
3,Conjugate Gradient,"[2.777777777777778, -0.22222222222222254, 1.1111111111111112]",2,True
